<a href="https://colab.research.google.com/github/BalthazazarDuc/Data-Science-Machine-Learning/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 36.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')


nlp = spacy.load('fr_core_news_sm')

def extract_features(sentence):
    doc = nlp(sentence)
    word_count = len([token.text for token in doc])
    unique_words = len(set(token.text for token in doc))
    comma_count = sentence.count(',')

    # Initialize verb tense counts
    present_tense_verbs = 0
    past_tense_verbs = 0
    future_tense_verbs = 0

    features = {
        'word_count': word_count,
        'unique_words': unique_words,
        'comma_count': comma_count,
    }
    return features


train_feature = pd.DataFrame(train['sentence'].apply(extract_features).tolist())
train_data = pd.concat([train, train_feature], axis=1)

test_feature = pd.DataFrame(test['sentence'].apply(extract_features).tolist())
test_data = pd.concat([test, test_feature], axis=1)

X = train_data.drop(['difficulty'], axis=1)  # 'difficulty' is the target feature
y = train_data['difficulty']

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('txt', TfidfVectorizer(ngram_range=(1, 2)), 'sentence'),  # Assuming the text column is named 'text_column'
        ('num', StandardScaler(), ['word_count', 'unique_words', 'comma_count'])  # Standardize numeric features
    ])


In [16]:
from sklearn.svm import SVC

clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', max_iter=10000))  # Use SVC with a linear kernel
])

In [20]:
clf.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('txt',
                                                  TfidfVectorizer(ngram_range=(1,
                                                                               2)),
                                                  'sentence'),
                                                 ('num', StandardScaler(),
                                                  ['word_count', 'unique_words',
                                                   'comma_count'])])),
                ('classifier', SVC(kernel='linear', max_iter=10000))])

In [21]:
predictions = clf.predict(test_data)

In [22]:
test['difficulty'] = predictions
result_extended = test.drop(columns=['sentence'])

result_extended.to_csv('result_pipeline.csv', index=False)